# Imports

In [1]:
import pyxdf
import pandas as pd
import numpy as np
import sounddevice as sd
from glob import glob
from tqdm import tqdm
import librosa
import datetime


# Load Data

In [2]:
sub_files = glob('../../../*/*.xdf')
sub_files

['../sub-P5318014/sub-P5318014_ses-S001_task-CUNY_run-001_mobi.xdf',
 '../sub-5182010/sub-P5182010_ses-S001_task-CUNY_run-001_mobi.xdf',
 '../sub-P5447527/sub-P5447527_ses-S001_task-CUNY_run-001_mobi.xdf',
 '../sub-5958030/sub-P5958030_ses-S001_task-CUNY_run-001_mobi.xdf',
 '../sub-P5548165/sub-P5548165_ses-S001_task-CUNY_run-001_MOBI.xdf']

In [3]:
xdf_path = sub_files[3]
data, header = pyxdf.load_xdf(xdf_path)
streams_collected = [stream['info']['name'][0] for stream in data]


'''
HELPERS
'''
def get_event_data(event, df, stim_df):
    return df.loc[(df.lsl_time_stamp >= stim_df.loc[stim_df.event == 'Onset_'+event, 'lsl_time_stamp'].values[0]) & 
                  (df.lsl_time_stamp <= stim_df.loc[stim_df.event == 'Offset_'+event, 'lsl_time_stamp'].values[0])]

def get_secs_between_triggers(trigger1, trigger2, stim_df):
    return stim_df.loc[stim_df.trigger == trigger1, 'time'].values[0] - stim_df.loc[stim_df.trigger == trigger2, 'time'].values[0]

                  
streams_collected

['WebcamStream',
 'Tobii',
 'OpenSignals',
 'EGI NetAmp 0',
 'Stimuli_Markers',
 'Microphone']

# Stimulus

In [ ]:
stim_dat = data[streams_collected.index('Stimuli_Markers')]
stim_df = pd.DataFrame(stim_dat['time_series'])
stim_df.rename(columns={0: 'trigger'}, inplace=True)
events = {
    200: 'Onset_Experiment',
    10: 'Onset_RestingState',
    11: 'Offset_RestingState',
    500: 'Onset_StoryListening',
    501: 'Offset_StoryListening',
    100: 'Onset_10second_rest',
    101: 'Offset_10second_rest', 
    20: 'Onset_CampFriend',
    21: 'Offset_CampFriend',
    30: 'Onset_FrogDissection',
    31: 'Offset_FrogDissection',
    40: 'Onset_DanceContest',
    41: 'Offset_DanceContest',
    50: 'Onset_ZoomClass',
    51: 'Offset_ZoomClass',
    60: 'Onset_Tornado',
    61: 'Offset_Tornado',
    70: 'Onset_BirthdayParty',
    71: 'Offset_BirthdayParty',
    300: 'Onset_subjectInput',
    301: 'Offset_subjectInput',
    302: 'Onset_FavoriteStory',
    303: 'Offset_FavoriteStory',
    304: 'Onset_WorstStory',
    305: 'Offset_WorstStory',
    400: 'Onset_impedanceCheck',
    401: 'Offset_impedanceCheck',
    80: 'Onset_SocialTask',
    81: 'Offset_SocialTask',
    201: 'Offset_Experiment',
}

story_onsets = [20, 30, 40, 50, 60, 70]

# relabel the event if the trigger is in the events dictionary, else if 
stim_df['event'] = stim_df['trigger'].apply(lambda x: events[x] if x in events.keys() else 'Bx_input')

# relabel the event as a psychopy timestamp if the trigger is greater than 5 digits
stim_df.loc[stim_df.trigger.astype(str).str.len() > 5, 'event'] = 'psychopy_time_stamp'
stim_df['lsl_time_stamp'] = stim_dat['time_stamps']
stim_df['time'] = (stim_dat['time_stamps'] - stim_dat['time_stamps'][0])/1000
stim_df

,trigger,event,lsl_time_stamp,time
0,2.000000e+02,Onset_Experiment,1.073920e+06,0.000000e+00
1,1.735058e+09,psychopy_time_stamp,1.073920e+06,9.799609e-09
2,1.000000e+01,Onset_RestingState,1.073940e+06,1.976684e-02
3,1.735058e+09,psychopy_time_stamp,1.073940e+06,1.976685e-02
4,1.100000e+01,Offset_RestingState,1.074240e+06,3.197662e-01
...,...,...,...,...
256,1.735060e+09,psychopy_time_stamp,1.076101e+06,2.180627e+00
257,2.010000e+02,Offset_Experiment,1.076106e+06,2.185632e+00
258,1.735060e+09,psychopy_time_stamp,1.076106e+06,2.185632e+00
259,4.000000e+00,Bx_input,1.076106e+06,2.185633e+00


# Microphone Data 

In [5]:
mic_data = data[streams_collected.index('Microphone')]
mic_df = pd.DataFrame(mic_data['time_series'], columns=['int_array'])
mic_df['bytestring'] = mic_df['int_array'].apply(lambda x: np.array(x).tobytes())
mic_df['time_stamps'] = (mic_data['time_stamps'] - mic_data['time_stamps'][0])/441000

mic_df['lsl_time_stamp'] = mic_data['time_stamps']
mic_df['time'] = (mic_df.lsl_time_stamp - mic_df.lsl_time_stamp[0])
mic_df['time_delta'] = mic_df['lsl_time_stamp'].diff()

mic_df.tail()

,int_array,bytestring,time_stamps,lsl_time_stamp,time,time_delta
84275096,-12,b'\xf4\xff\xff\xff\xff\xff\xff\xff',0.004333,1.076166e+06,1911.047610,0.000023
84275097,-9,b'\xf7\xff\xff\xff\xff\xff\xff\xff',0.004333,1.076166e+06,1911.047633,0.000023
84275098,8,b'\x08\x00\x00\x00\x00\x00\x00\x00',0.004333,1.076166e+06,1911.047655,0.000023
84275099,-3,b'\xfd\xff\xff\xff\xff\xff\xff\xff',0.004333,1.076166e+06,1911.047678,0.000023
84275100,33,b'!\x00\x00\x00\x00\x00\x00\x00',0.004333,1.076166e+06,1911.047701,0.000023


In [6]:
mic_data

{'info': defaultdict(list,
             {'name': ['Microphone'],
              'type': ['AudioCapture'],
              'channel_count': ['1'],
              'channel_format': ['int16'],
              'source_id': ['audio-C4-BIOSIGNALS'],
              'nominal_srate': ['44100.00000000000'],
              'version': ['1.100000000000000'],
              'created_at': ['3102122.804696500'],
              'uid': ['9debbde9-379a-4bac-8c08-1abf436665f1'],
              'session_id': ['default'],
              'hostname': ['C4-BIOSIGNALS'],
              'v4address': [None],
              'v4data_port': ['16573'],
              'v4service_port': ['16573'],
              'v6address': [None],
              'v6data_port': ['16573'],
              'v6service_port': ['16573'],
              'desc': [None],
              'stream_id': 4,
              'effective_srate': 44098.89924003096}),
 'footer': {'info': defaultdict(list,
              {'first_timestamp': ['3104052.665603676'],
               

In [7]:
nom_srate = float(mic_data['info']['nominal_srate'][0])
srate = mic_data['info']['effective_srate']
nom_srate, srate

(44100.0, 44098.89924003096)

## Compare wav and LSL lengths

In [8]:
# get wav file
wav_files = glob('../../../*/*.wav')
wav_files

wav_path = wav_files[0]
wav_path

'../sub-P5318014/sub-P5318014_task-CUNY_run-001_mic.wav'

In [9]:
# calculate wav and lsl durations
wav_dur = round(librosa.get_duration(path=wav_path), 2)
lsl_dur = round(mic_df['lsl_time_stamp'].iloc[-1]- mic_df['lsl_time_stamp'].iloc[0], 2)

# convert to hh:mm:ss
wav_dt = datetime.timedelta(seconds=wav_dur)
wav_dur_dt = str(datetime.timedelta(seconds=round(wav_dt.total_seconds())))
lsl_dt = datetime.timedelta(seconds=lsl_dur)
lsl_dur_dt = str(datetime.timedelta(seconds=round(lsl_dt.total_seconds())))


# make df of durations
variables = {name: [globals()[name], globals()[name+'_dt']]  for name in ['wav_dur', 'lsl_dur']}
durations_df = pd.DataFrame.from_dict(variables, orient = 'index', columns = ['seconds', 'mm:ss'])
print(durations_df.to_string() + '\n')

# print if they are close or not (margin on 5 seconds rn)
if abs(wav_dur - lsl_dur) > 5:
    print(f'Warning: The duration of the wav file ({wav_dur}) and the duration of the lsl time stamps ({lsl_dur}) differ by more than 5 seconds. \n')
    if wav_dur > lsl_dur:
        print('The wav file is longer than the lsl time stamps by {} seconds'.format(wav_dur - lsl_dur))
    if lsl_dur > wav_dur:
        print('The lsl time stamps are longer than the wav file by {} seconds'.format(lsl_dur - wav_dur))
else: 
    print('The wav file and the lsl time stamps are the same length!')

         seconds    mm:ss
wav_dur  1989.40  0:33:09
lsl_dur  1911.05  0:31:51


The wav file is longer than the lsl time stamps by 78.35000000000014 seconds


## Slice parts we care about

In [10]:
slice_mic1 = mic_df.loc[(mic_df.lsl_time_stamp >= stim_df.loc[stim_df.event == 'Onset_StoryListening', 'lsl_time_stamp'].values[0]) & 
                  (mic_df.lsl_time_stamp <= stim_df.loc[stim_df.event == 'Onset_impedanceCheck', 'lsl_time_stamp'].values[0])]
slice_mic2 = mic_df.loc[(mic_df.lsl_time_stamp >= stim_df.loc[stim_df.event == 'Offset_impedanceCheck', 'lsl_time_stamp'].values[0]) & 
                  (mic_df.lsl_time_stamp <= stim_df.loc[stim_df.event == 'Offset_Experiment', 'lsl_time_stamp'].values[0])]

slice_exp1 = stim_df.loc[(stim_df.lsl_time_stamp >= stim_df.loc[stim_df.event == 'Onset_StoryListening', 'lsl_time_stamp'].values[0]) & 
                  (stim_df.lsl_time_stamp <= stim_df.loc[stim_df.event == 'Onset_impedanceCheck', 'lsl_time_stamp'].values[0])]
slice_exp2 = stim_df.loc[(stim_df.lsl_time_stamp >= stim_df.loc[stim_df.event == 'Offset_impedanceCheck', 'lsl_time_stamp'].values[0]) & 
                  (stim_df.lsl_time_stamp <= stim_df.loc[stim_df.event == 'Offset_Experiment', 'lsl_time_stamp'].values[0])]

## Compare experiment and mic durations with sliced data

In [11]:
slice_mic = mic_df.loc[(mic_df.lsl_time_stamp >= stim_df.loc[stim_df.event == 'Onset_StoryListening', 'lsl_time_stamp'].values[0]) & 
                  (mic_df.lsl_time_stamp <= stim_df.loc[stim_df.event == 'Offset_Experiment', 'lsl_time_stamp'].values[0])]

slice_exp = stim_df.loc[(stim_df.lsl_time_stamp >= stim_df.loc[stim_df.event == 'Onset_StoryListening', 'lsl_time_stamp'].values[0]) & 
                  (stim_df.lsl_time_stamp <= stim_df.loc[stim_df.event == 'Offset_Experiment', 'lsl_time_stamp'].values[0])]

In [15]:
exp_dur0-mic_dur0

15.48070000000007

In [13]:
mic_start = slice_mic['lsl_time_stamp'].values[0]
mic_stop = slice_mic['lsl_time_stamp'].values[-1]
mic_dur0 = round(mic_stop - mic_start, 4)
mic_dur0

1850.3854

In [14]:
exp_start = slice_exp['lsl_time_stamp'].values[0]
exp_stop = slice_exp['lsl_time_stamp'].values[-1]
exp_dur0 = round(exp_stop - exp_start, 4)
exp_dur0

1865.8661

In [14]:
slice_mic.head()

,int_array,bytestring,time_stamps,lsl_time_stamp,time,time_delta
844833,21,b'\x15\x00\x00\x00\x00\x00\x00\x00',0.000043,168540.148158,19.157535,0.000023
844834,22,b'\x16\x00\x00\x00\x00\x00\x00\x00',0.000043,168540.148181,19.157557,0.000023
844835,20,b'\x14\x00\x00\x00\x00\x00\x00\x00',0.000043,168540.148204,19.157580,0.000023
844836,23,b'\x17\x00\x00\x00\x00\x00\x00\x00',0.000043,168540.148226,19.157603,0.000023
844837,20,b'\x14\x00\x00\x00\x00\x00\x00\x00',0.000043,168540.148249,19.157625,0.000023


In [15]:
exp_stop
print(exp_start, mic_start)

168540.1481515094 168540.1481582377


In [16]:
sd.play(slice_mic.int_array[:1000])

In [17]:
mic_start1 = slice_mic1['lsl_time_stamp'].values[0]
mic_stop1 = slice_mic1['lsl_time_stamp'].values[-1]
mic_dur1 = round(mic_stop1 - mic_start1, 4)

mic_start2 = slice_mic2['lsl_time_stamp'].values[0]
mic_stop2 = slice_mic2['lsl_time_stamp'].values[-1]
mic_dur2 = round(mic_stop2 - mic_start2, 4)
mic_dur = mic_dur1 + mic_dur2
mic_dur

1491.7426

In [18]:
mic_dur1

1151.5771

In [19]:
mic_dur2

340.1655

In [20]:
exp_start1 = slice_exp1['lsl_time_stamp'].values[0]
exp_stop1 = slice_exp1['lsl_time_stamp'].values[-1]
exp_dur1 = round(exp_stop1 - exp_start1, 4)

exp_start2 = slice_exp2['lsl_time_stamp'].values[0]
exp_stop2 = slice_exp2['lsl_time_stamp'].values[-1]
exp_dur2 = round(exp_stop2 - exp_start2, 4)
exp_dur = exp_dur1 + exp_dur2
exp_dur

1546.2329

In [21]:
exp_dur1

1206.0674

In [22]:
exp_dur2

340.1655